## imports

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [47]:
cd /content/drive/My Drive/BIO/BIO PROJECT 2

/content/drive/My Drive/BIO/BIO PROJECT 2


In [48]:
!pip install markov_clustering

In [0]:
import markov_clustering as mc
import networkx as nx
import random
import pandas as pd
from tqdm import tqdm
from scipy.stats import hypergeom

## 2.2 Apply clustering methods for disease modules discovery

In [90]:
# importing list of seed genes we found before
seed_gs = pd.read_csv('task_1.1c.tab',sep="\t")
seed_gs = seed_gs.drop('Unnamed: 0',axis=1)
seed_gs = seed_gs.dropna(subset=['geneSymbol'])
seed_genes_list = list(seed_gs.geneSymbol)
seed_genes_list.remove('DLEU2') # not presented
print('Number of seed genes: ', len(seed_genes_list))

Number of seed genes:  107


In [0]:
# reading files
#LCC_U = pd.read_csv('LCC_U_local_measures', delimiter='\t', usecols=['Name'])['Name'].tolist()
#LCC_I = pd.read_csv('LCC_I', delimiter='\t', usecols=['Name'])['Name'].tolist()
intersection_interactome = pd.read_csv('intersection_interactome', delimiter='\t',\
                                       usecols=['symbol_a','symbol_b'])
union_interactome = pd.read_csv('union_interactome', delimiter='\t',\
                                       usecols=['symbol_a','symbol_b'])

In [92]:
# recreating graphs for LCC_U and LCC_I

def graphs_from_edges(df):
    _edges = list()
    for i in tqdm(range(len(df['symbol_a']))):
        #if df['uniprot_a'][i] in _nodes_list:
           # if df['uniprot_b'][i] in _nodes_list:
        _edges.append((df['symbol_a'][i], df['symbol_b'][i]))
    return(_edges)

LCC_U_edges = graphs_from_edges(union_interactome)
LCC_I_edges = graphs_from_edges(intersection_interactome)

100%|██████████| 533078/533078 [00:11<00:00, 45821.02it/s]


In [0]:
# Cluster I-LCC and U-LCC using the MCL algorithm to get the modules

def putative_disease_modules(_edges_list, filename):
    # creating graph from list of edges
    G = nx.Graph()
    G.add_edges_from(_edges_list)
    # renaming nodes to integers
    G_int = nx.convert_node_labels_to_integers(G,label_attribute='old_label')
    # clustering graph using MCL algo
    matrix_G = nx.to_scipy_sparse_matrix(G_int) # get the adjacency matrix (in sparse form)
    result_G = mc.run_mcl(matrix_G, inflation=2)  # run MCL with default parameters
    clusters_G = mc.get_clusters(result_G) # get clusters
    clusters_candidates = [cluster for cluster in clusters_G if len(cluster) >= 10]

    # creating map from integer id to label of nodes
    nodes_id_label_map = dict()
    for elem in list(G_int.nodes(data=True)):
        nodes_id_label_map.update({elem[1]['old_label']:elem[0]})
    # mapping seed genes into integer id
    seed_genes_list_id = [nodes_id_label_map[key] for key in seed_genes_list]

    # invert mapping to integer-key, label-value
    nodes_label_id_map = {v: k for k, v in nodes_id_label_map.items()}

    # finding clusters igger than 10 nodes, containing at least 1 seed gene
    clusters_information = []
    clusters_ = []
    for cluster in clusters_candidates:
        if len(set(cluster).intersection(set(seed_genes_list_id))) > 0:
            clusters_.append(cluster)
            # clusters_information - # of seed genes, total # of genes, seed genes ids
            clusters_information.append([len(set(cluster).intersection(set(seed_genes_list_id))),\
                                        len(cluster), set(cluster).intersection(set(seed_genes_list_id))])
    
    f = open(filename + '_putative_disease_modules' + '.txt', "w")
    f.write('Number of clusters containing seed genes and no. of nodes >= 10: '\
            + str(len(clusters_)) + '\n' + '\n')
    
    f.write('Putative disease modules:' + '\n' + '\n')
    # hypergeometric test
    for i in range(len(clusters_)):
        M = len(G_int.nodes) # population size = size of graph
        n = len(seed_genes_list_id) # number of successes in population = number of seed genes
        N = clusters_information[i][1] # sample size = size of module[i]/cluster[i]
        x = clusters_information[i][0] # number of successes in sample
        if hypergeom.sf(x-1, M, n, N) < 0.05:
            f.write('    Module ID: ' + str(i) + '\n')
            f.write('Clustering algorithm: MCL' + '\n')
            f.write('No. of seed genes in module: ' + str(clusters_information[i][0]) + '\n')
            f.write('Total no. of genes in module: ' + str(clusters_information[i][1]) + '\n')
            seed_genes = [nodes_label_id_map[id] for id in list(clusters_information[i][2])]
            f.write('Seed genes IDs: ' + str(seed_genes) + '\n')
            genes = [nodes_label_id_map[id] for id in clusters_[i]]
            f.write('Genes IDs: '+ str(genes) + '\n')
            f.write('p-val: ' +  str(round(hypergeom.sf(x-1, M, n, N),4)) + '\n' + '\n')
            f1 = open(filename + '_cluster_' + str(i) + '.txt', "w")
            for g in genes:
                f1.write(g + '\n')
            f1.close()

    f.close()

In [0]:
putative_disease_modules(LCC_U_edges, 'U_LCC')

In [0]:
putative_disease_modules(LCC_I_edges, 'I_LCC')